In [1]:
import pandas as pd
import json
from scipy.spatial.distance import cdist
import numpy as np
import csv

In [2]:
with open('data/original/eco.json') as json_file:
    eco_json = json.load(json_file)

In [3]:
mobile_station_data = pd.read_csv('data/tranformed/data-49158-2020-10-16.csv', 
                                  index_col='Unnamed: 0')

In [4]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

In [5]:
# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

In [6]:
data_list = []
check_points = mobile_station_data[['Latitude_WGS84', 'Longitude_WGS84']].values
for current_element in eco_json:
    if (len(current_element[0]) % 2) != 0:
        data_list.append({'Latitude':current_element[0][-3], 
                     'Longitude':current_element[0][-2], 
                     'Distance':current_element[0][-1], 
                     'Icon':current_element[1], 
                     'Description':current_element[2]})
    else:
        current_closest_point = closest_point([current_element[0][0], current_element[0][1]], 
                                      check_points)
        distance = haversine(current_element[0][0], 
                        current_element[0][1], 
                        current_closest_point[0],
                        current_closest_point[1],
                        to_radians=True
                       ) * 1000
        if distance > 5000:
            distance = 5000
        data_list.append({'Latitude':current_element[0][0], 
             'Longitude':current_element[0][1], 
             'Distance':distance, 
             'Icon':current_element[1], 
             'Description':current_element[2]})

In [7]:
data = pd.DataFrame(data_list,columns=['Latitude', 'Longitude', 'Distance', 'Icon', 'Description'])


In [8]:
data

,Latitude,Longitude,Distance,Icon,Description
0,56.478928,38.044710,5000.000000,q,<b>ФГУП «Радон»</b><br>Утилизация радиоактивны...
1,55.739785,37.573154,475.775405,q,<b>ФГУП «Радон»</b><br>Утилизация радиоактивны...
2,55.827606,37.413436,353.983568,q,<b>ФГУП «Радон»</b><br>Служба радиационно-авар...
3,55.898662,37.521513,285.736976,q,<b>ФГУП «Радон»</b><br>Утилизация радиоактивны...
4,55.691169,37.778319,722.609855,q,<b>ФГУП «Радон»</b><br>Утилизация радиоактивны...
...,...,...,...,...,...
303,55.800410,38.913880,5000.000000,c,<b>Свалка химических отходов</b><br> бывшего з...
304,55.678419,37.441000,914.286862,w,<b>Полигон захоронения отходов от разборки пят...
305,56.397644,37.520986,5000.000000,o,<a href='http://www.ilimgroup.ru/'><b>Илим Гоф...
306,55.564660,39.576273,5000.000000,o,<a href='http://www.shatura.com'><b>Шатурский ...


In [9]:
mobile_station_data

,Date,global_id,AdmArea,District,Longitude_WGS84,Latitude_WGS84,Location,Results,geoData,Rank
0,11.01.2016,272626552,Восточный административный округ,район Восточный,37.797262,56.007634,"посёлок Акулово, дом 7",Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.797261999821, 56....",0
1,11.01.2016,272626555,Восточный административный округ,район Новогиреево,37.801156,55.743139,"Перовская улица, дом 66, корпус 1",Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.801155999846, 55....",0
2,13.01.2016,272626556,Зеленоградский административный округ,район Старое Крюково,37.213224,55.978150,"г. Зеленоград, Георгиевский проспект, дом 7",Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.213224000395, 55....",0
3,13.01.2016,272626557,Зеленоградский административный округ,район Силино,37.168481,55.992985,"город Зеленоград, корпус 1131",Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.168481000184, 55....",0
4,13.01.2016,272626558,Зеленоградский административный округ,район Савёлки,37.255253,55.991013,"город Зеленоград, корпус 526",Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.255252999689, 55....",0
...,...,...,...,...,...,...,...,...,...,...
4053,16.08.2020,1058872868,Юго-Восточный административный округ,район Некрасовка,37.927779,55.686284,"2-я Вольская улица, дом 26",По отдельным веществам в атмосферном воздухе в...,"{type=Point, coordinates=[37.927779, 55.686284]}",8
4054,02.09.2019,1019525010,Западный административный округ,район Очаково-Матвеевское,37.453827,55.706934,"Аминьевское шоссе, дом 17",По отдельным веществам в атмосферном воздухе в...,"{type=Point, coordinates=[37.453827, 55.706934]}",7
4055,27.11.2019,1026704276,Северо-Восточный административный округ,район Южное Медведково,37.629430,55.878329,"улица Молодцова, дом 23, корпус 1",По отдельным веществам в атмосферном воздухе в...,"{type=Point, coordinates=[37.62943, 55.878329]}",7
4056,25.06.2020,1045811698,Северо-Восточный административный округ,район Лианозово,37.571107,55.901521,Лианозовский парк,Содержание загрязняющих веществ в атмосферном ...,"{type=Point, coordinates=[37.571107, 55.901521]}",0


In [10]:
data.to_csv('data/final/result.csv', sep=';', quoting=csv.QUOTE_NONNUMERIC, index=False)